# Energy production & consumption forecast

This project compiles many time series forecasting models and compares their performances. The models introduced are :

*   ARIMA
*   ARIMAX
*   Prophet (Facebook)
*   LSTM
*   Chronos (Amazon)

Done using the Our World in Data Energy dataset : https://github.com/owid/energy-data?tab=readme-ov-file


## Imports & prerequisits

In [ ]:
!git clone https://github.com/theophile-bb/Time-series-predictions-on-energy-production.git

In [ ]:
%cd Time-series-predictions-on-energy-production

In [ ]:
pip install -r requirements.txt

In [ ]:
import sys
sys.path.append("src")

In [ ]:
# Utils

from utils import *

In [ ]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
path = "data/energy_sample.json"

***

Let's start by reading the data from the json and putting the dataframe into shape :

In [ ]:
data = load_json(path)

In [ ]:
df = create_df(data)

In [ ]:
df.head()

***

In [ ]:
figs = []

## Production Data

The goal here is to proceed manualy to try and forecast the production for one of the energy sources.

To do so we'll try multiple forecasting models and see how they perform to forecast with our data.

As a starter we'll begin by sticking to only the USA and we'll study the oil, gas and coal productions since year 1900.

In [ ]:
countries = ["United States"]
col = ['region', 'coal_production', 'oil_production', 'gas_production']

df_country = get_countries(df, countries)
df_production = get_columns(df_country, col)

In [ ]:
plot_total_production = plot_graph(df_production,"Production over time")
figs.append(plot_total_production)

### ARIMA

In [ ]:
model_name = "ARIMA"

The ARIMA models combine two models and one method. These are:

*   Auto Regression(AR)
*   Moving Average(MA)
*   Differencing for stationarity(I)

A nonseasonal ARIMA model is classified as an "ARIMA(p,d,q)" model, where: p is the number of autoregressive terms(AR), d is the number of nonseasonal differences needed for stationarity(I), and q is the number of lagged forecast errors in the prediction equation(MA).

One of the preprocessing steps is to determine the optimal orders **(p, d, q)** of our ARIMA model. The simplest one is the order of differencing d as this can be verified by carrying out a statistical test for stationarity. The most popular one is the Augmented Dickey-Fuller (ADF), where the null hypothesis is that the time series is not stationary.

d refers to the number of differencing transformations required by the time series to get stationary. So we can use pandas 'diff()' function once or more and recall stationarity function to find the d-value.  If the P-value of Dickey-Fuller test is less than 0.05, the column is stationary, otherwise it is not stationary.

In [ ]:
col = 'coal_production'

The function stationarity will ease the process by returning d (number of differenciations) and the associated P-value.

In [ ]:
diff_col, d = stationarity(df_production[col])

The autoregressive and moving-average orders (p,q) can be deduced by analysing the partial autocorrelation function (PACF) and autocorrelation function respectively. The gist of of this method is to plot a correlogram of the various lags/forecast errors of the time series to determine which are statistically significant.

In [ ]:
plot_autocorrelation = plot_acf(diff_col)
plot_partial_autocorrelation = plot_pacf(diff_col)

figs.append(plot_autocorrelation)
figs.append(plot_partial_autocorrelation)

We can use the Autocorrelation plot to get q and the Partial Autocorrelation one to get p.

We then split our dataset into training and testing set :

In [ ]:
train, test = split_data(df_production, 0.8)

We define the model with the parameters p, d, q. If p and q are equal to 0 it gives us an ARIMA(0, 1, 0). It then becomes an ARMA(0, 0) when differenced, which is random, uncorrelated, noise (random walk). The sum of noise terms has a mean of 0 : the expected position is still the starting point, but the variance around it increases over time.

In [ ]:
model_fit = train_ARIMA(train, col, 0,d,0)

We use the model to forecast on the test set

In [ ]:
test_forecast = predict_ARIMA(model_fit, test)

Finally we can plot the predictions on the test set and compare it with the actual ones :

In [ ]:
title = f"Production and prediction with {model_name} on Test set"
plot_ARIMA_test = plot_graph(test_forecast, title, ['coal_production','forecast'])
figs.append(plot_ARIMA_test)

Note : We can also use auto_arima to find the optimal p, d, q for our model, which we will use in the coming part : ARIMAX.

***

### ARIMAX

In [ ]:
model_name = "ARIMAX"

The **ARIMAX (AutoRegressive Integrated Moving Average with Exogenous Variables)** model is an extension of ARIMA that incorporates exogenous variables (X variables) to improve forecasting accuracy. It is a good model to use when there is an external factor (exogenous variable) influencing the time series.

First we'll test the model by training it on a training set and testing it with a test set. We are going to partition both the endogenous and exogenous values.

In [ ]:
endog_col = col
exog_col = ['gdp', 'population', 'primary_energy_consumption', 'oil_production', 'gas_production']

endog, exog = initialize_endog_exog(df_country,endog_col, exog_col)

In [ ]:
endog_train, endog_test = split_data(endog, 0.8)
exog_train, exog_test = split_data(exog, 0.8)

model = train_SARIMAX(endog_train, exog_train)

results = predict_SARIMAX(model, exog_test, len(exog_test))

In [ ]:
results_df = pd.DataFrame(data={'actual': list(endog_test), 'predicted': list(results)}, index=endog_test.index)

title = f"Production and prediction with {model_name} on Test set"
plot_ARIMAX_test = plot_graph(results_df, title , ['actual','predicted'])
figs.append(plot_ARIMAX_test)

We are now going to try and predict the future values for the coming years. To do so we'll need the exogenous values for these years.

We are going to approximate each of the future exogneous values (column) by using a simple linear regression with polynomial features. This model isn't optimal but it is quite simple and very well fitting for values with an exponential growth such as 'gdp' or 'population'.

Note that we use the **ReLu (Rectified Linear Unit)** function to prevent negative predictions.

In [ ]:
exog_pred = predict_exog(exog)

Now that we have our predicted exogenous values we can train the ARIMAX with the endogenous and exogenous values and then predict using the predicted exogenous values.

In [ ]:
model = train_SARIMAX(endog, exog)

results = predict_SARIMAX(model, exog_pred, len(exog_pred))

In [ ]:
data_plot = format_data_plot(endog)
forecast_plot = format_forecast_plot(results, df_production, len(results))

In [ ]:
plot_ARIMAX_forecast = plot_forecasts(data_plot, forecast_plot, col, model_name)
figs.append(plot_ARIMAX_forecast)

***

### Prophet

In [ ]:
model_name = "Prophet"

Prophet is an additive time series forecasting model that’s designed to work well with data that has:

*   Seasonality (daily, weekly, yearly patterns).
*   Trends (growth, decline, plateaus).
*   Holidays/Special Events (big spikes or dips).

It’s built to be robust to missing data and outliers, which is a common headache with other models.

In [ ]:
prophet_df = format_Prophet(df_country, col)

We disable seasonality because we have no use of it.

In [ ]:
period = 10

model = fit_Prophet(prophet_df)
results = predict_Prophet(model, period)

In [ ]:
results

We can try to predict for the next 10 years :

In [ ]:
data_plot = format_data_plot(df_production[col])
forecast_plot = format_Prophet_plot(df_production, results, period)

In [ ]:
plot_prophet_forecast = plot_forecasts(data_plot, forecast_plot, col, model_name)
figs.append(plot_prophet_forecast)

***

## LSTM

In [ ]:
model_name = "LSTM"

Long Short-Term Memory is a type of Recurrent Neural Network (RNN). LSTM learns from past values and patterns directly which makes it good when past values influence future values.

Regular RNNs struggle with long-term dependencies whereas the LSTM has a memory cell that helps retain information over long periods.

It decides what to remember and what to forget using three gates:

*   Forget Gate	: Decides what past info to forget.
*   Input Gate : Decides what new info to store.
*   Output Gate	: Decides what to output based on memory.

LSTM remembers long-term trends but adapts when patterns change.

###  Multi-step-ahead forecast

We predict multiple future steps directly on the entire horizon (e.g., next 5 years) at once.

In [ ]:
  scaled_data, scaler = scale_data(df_production, col)
  train, test = split_data(scaled_data, 0.8)

In [ ]:
seq_length = 3
horizon = 10

In [ ]:
model = fit_LSTM(train, test, seq_length, horizon)

results = flatten_prediction(scaled_data, seq_length, model, scaler)

In [ ]:
data_plot = format_data_plot(df_production[col])
forecast_plot = format_forecast_plot(results, df_production, len(results))

In [ ]:
plot_LSTM_forecast = plot_forecasts(data_plot, forecast_plot, col, model_name)
figs.append(plot_LSTM_forecast)

***

### Chronos Bolt

In [ ]:
model_name = "Chronos"

# *Note : Requires Python 3.10 to work*

Chronos-Bolt is a family of pretrained time series forecasting models which can be used for zero-shot forecasting. It is based on the T5 encoder-decoder architecture and has been trained on nearly 100 billion time series observations. It chunks the historical time series context into patches of multiple observations, which are then input into the encoder. The decoder then uses these representations to directly generate quantile forecasts across multiple future steps—a method known as direct multi-step forecasting.

References :

https://huggingface.co/autogluon/chronos-bolt-small

https://github.com/amazon-science/chronos-forecasting/tree/main


Let's start by training a chronos with a train set and see its performances on a test set

In [ ]:
ChronosData = format_Chronos(df_production, col)

train, test = split_data(ChronosData, 0.8)

period = len(test)
model = fit_Chronos(train, period)

print(ChronosData.index.get_level_values("timestamp").freq)

results_train = predict_Chronos(model, train)

In [ ]:
test['forecast'] = results_train['mean']
test["year"] = test.index.get_level_values("timestamp").year
test = test.reset_index()

In [ ]:
title = f"Production and prediction with {model_name} on Test set"
plot_chronos_test = plot_graph(test, title, ['target','forecast'])
figs.append(plot_chronos_test)

Then we can forecast for future years

In [ ]:
period = 10
model = fit_Chronos(ChronosData, period)
results = predict_Chronos(model, ChronosData)

In [ ]:
data_plot = format_data_plot(df_production[col])
forecast_plot = format_forecast_plot(results['mean'], df_production, period)

In [ ]:
plot_chronos_forecast = plot_forecasts(data_plot, forecast_plot, col, model_name)
figs.append(plot_chronos_forecast)

***

In [ ]:
save_figs(figs, folder="plots")